In [1]:
import findspark
findspark.init('/home/dangkhoa/spark-2.3.1-bin-hadoop2.7')

## Session

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Linear_Regression').getOrCreate()

## Load dataset

In [3]:
raw_data = spark.read.csv("Ecommerce_Customers.csv",inferSchema=True,header=True)

raw_data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [4]:
raw_data.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

## Vector Assembler
- Grab all columns -> 1 single vector

In [5]:
raw_data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [6]:
from pyspark.ml.feature import VectorAssembler

# Define assembler 
assembler = VectorAssembler(
    inputCols=['Avg Session Length',
               'Time on App', 
               'Time on Website',
               'Length of Membership'],
    outputCol='features')

# transform
vector_indexed_data = assembler.transform(raw_data)
vector_indexed_data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [7]:
vector_indexed_data.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

## Create dataset - Train/Test set

In [8]:
# X = features, y = Yearly Amount Spent
dataset = vector_indexed_data.select('features', 'Yearly Amount Spent')

dataset.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows



In [9]:
train_data,test_data = dataset.randomSplit([0.7,0.3])

## Linear Regression

In [10]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    featuresCol='features',
    labelCol='Yearly Amount Spent',
    predictionCol='prediction')

In [11]:
# Train
model = lr.fit(train_data)

## Model Summary

In [12]:
print("Coefficients: {}\nIntercept: {}".format(model.coefficients,model.intercept))

Coefficients: [25.716831307604348,38.77131232830294,0.348391002838057,62.023176370878254]
Intercept: -1050.022026531478


In [13]:
trainingSummary = model.summary

# train RMSE, MSE
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("MSE: {}".format(trainingSummary.meanSquaredError))

RMSE: 9.895783796874104
MSE: 97.92653695447608


## Evaluate Testset

In [14]:
test_results = model.evaluate(test_data)

# test RMSE, MSE
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 10.030133309924013
MSE: 100.60357421484723


## Prediction on Testset

In [15]:
X = test_data.select('features')

predictions = model.transform(X)

In [16]:
# Predict
predictions.show(5)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 398.4512751354207|
|[30.7377203726281...| 451.2734192522885|
|[30.9716756438877...| 488.4169294960441|
|[31.1239743499119...|509.02894055039974|
|[31.1280900496166...| 565.2856145898918|
+--------------------+------------------+
only showing top 5 rows



In [17]:
# Real label
test_data.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[29.5324289670579...|  408.6403510726275|
|[30.7377203726281...|  461.7807421962299|
|[30.9716756438877...|  494.6386097568927|
|[31.1239743499119...|  486.9470538397658|
|[31.1280900496166...|  557.2526867470547|
+--------------------+-------------------+
only showing top 5 rows

